# Dynamic SQL and Sessions 

This notebook goes along with my blog post:

## Example 1:

Compare how sp\_executesql works as opposed to xp\_cmdshell

In [10]:
SET NOCOUNT ON

Commands completed successfully.

Total execution time: 00:00:00.001

In [11]:
SELECT @@SPID as SpidFromBatch

DECLARE @Sql NVARCHAR(MAX) = 'SELECT @@SPID as SpidFromDynamicSQL'
EXEC sys.sp_executesql @Sql

EXEC master..xp_cmdshell 'sqlcmd -S"localhost\sql2019" -Q"SELECT @@SPID as SpidFromSQLCmd"'

Commands completed successfully.

Total execution time: 00:00:00.111

SpidFromBatch
82


SpidFromDynamicSQL
82


output
SpidFromSQLCmd
--------------
74
NULL
(1 rows affected)
NULL


## Example 2:

Show how dynamic SQL works with temp tables and variables

In [12]:
DROP TABLE IF EXISTS #Test
/* Use this temp table for the examples */
CREATE TABLE #Test
(ID int IDENTITY(1,1),
SampleValue varchar(50))


Commands completed successfully.

Total execution time: 00:00:00.007

In [13]:
/* How I usually think of doing the statements */
DECLARE @Sql nvarchar(100)

SELECT @Sql = 'SELECT ''TestValue from standalone dynamic statement'' as TestValue'

INSERT INTO #Test (SampleValue)
EXEC sp_executesql @Sql

SELECT * FROM #Test

Commands completed successfully.

Total execution time: 00:00:00.012

ID,SampleValue
1,TestValue from standalone dynamic statement


In [14]:
/* How I can think of doing the statements */
DECLARE @Sql nvarchar(100)

SELECT @Sql = '
INSERT INTO #Test (SampleValue)
SELECT ''TestValue from combined dynamic statement'' as TestValue'

EXEC sp_executesql @Sql

SELECT * FROM #Test

Commands completed successfully.

Total execution time: 00:00:00.006

ID,SampleValue
1,TestValue from standalone dynamic statement
2,TestValue from combined dynamic statement
